## Packages

In [1]:
here::i_am("code/VADER.ipynb")
library(here)
library(tidyr)
library(purrr)
library(dplyr)
devtools::load_all("~/Code/R/ktools/")

here() starts at /Users/knguyen/Documents/WIP/Papers/Astrid


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


ℹ Loading ktools
ℹ Re-compiling ktools (debug build)



── R CMD INSTALL ───────────────────────────────────────────────────────────────
* installing *source* package ‘ktools’ ...
** using staged installation
** libs
* DONE (ktools)


Only needed for German

In [2]:
Sys.getlocale()
Sys.setlocale("LC_ALL", "de_DE.UTF-8")
Sys.getlocale()

[1] "C/UTF-8/C/C/C/C"

[1] "de_DE.UTF-8/de_DE.UTF-8/de_DE.UTF-8/C/de_DE.UTF-8/C"

[1] "de_DE.UTF-8/de_DE.UTF-8/de_DE.UTF-8/C/de_DE.UTF-8/C"

Read forum data

In [3]:
forum <- readRDS(here("data/forum_emj.rds"))
glimpse(forum)
forum$text2[1]

Rows: 11,208
Columns: 9
$ user        <chr> "BRIGITTE Community-Team", "BRIGITTE Community-Team", "Ina…
$ date        <chr> "02.06.2020, ", "02.06.2020, ", "02.06.2020, ", "03.06.202…
$ time        <hms> 48240 secs, 55140 secs, 66540 secs, 5940 secs, 25560 secs,…
$ mid         <chr> "post_message_34705014", "post_message_34705172", "post_me…
$ text        <chr> "Liebe User*innen, neben Fragen der Kinderbetreuung, Kitas…
$ user_quoted <chr> NA, NA, NA, NA, NA, "['BRIGITTE Community-Team']", NA, "['…
$ quoted_text <chr> NA, NA, NA, NA, NA, "['- Bitte verzichten Sie darauf, in d…
$ id          <chr> "34705014", "34705172", "34705352", "34705824", "34705866"…
$ text2       <chr> "Liebe User*innen, neben Fragen der Kinderbetreuung, Kitas…


Liebe User*innen, neben Fragen der Kinderbetreuung, Kitas oder Home-Schooling soll hier auch Platz für weitere Familienthemen wie z.B. den Umgang mit An gehörigen in Seniorenheimen sein. Hier können Sie sehen, welchen bisherigen Themen in diesem neuen Thema aufgehen Wir wünschen Ihnen spannende Diskussionen. Viele Grüße vom BRIGITTE Community-Team 
"Liebe User*innen, neben Fragen der Kinderbetreuung, Kitas oder Home-Schooling soll hier auch Platz für weitere Familienthemen wie z.B. den Umgang mit An gehörigen in Seniorenheimen sein. Hier können Sie sehen, welchen bisherigen Themen in diesem neuen Thema aufgehen Wir wünschen Ihnen spannende Diskussionen. Viele Grüße vom BRIGITTE Community-Team"

## Split German text before translating?

> See below, DeepL seems better at splitting sentences as well.

In [ ]:
library("spacyr")
Sys.setenv(RETICULATE_PYTHON = "/Users/knguyen/opt/miniconda3/bin/python")
spacy_initialize('de_dep_news_trf')

Manually split German with `spacyr`

In [88]:
spacy_tokenize(forum$text2[100], "sentence")
quanteda::tokenize_sentence(forum$text2[100])

$...
[1] "Was mich etwas besorgt ist, dass es in Isreal (was ja sehr schnell und stark mit Einschränkungen reagiert hat) jetzt anscheinend wieder zu vermehrten Infektionen in Schulen zu kommen scheint."
[2] "Die sind jetzt wohl in einem nervigen \"auf und zu\" von Schulen."                                                                                                                              
[3] "Quelle: SZ, 5.06.20, S. 16, Alexandra Föderl-Schmid, \"Schule auf, Schule zu\""                                                                                                                 
[4] "Sie reagieren einerseits mit den erwähnten Schließungen und mit sog. Sentinels (also Stichproben in ausgewählten Einrichtungen)."                                                               
[5] "Und mit einem Antikörper Screenings mit 70000 Personen in der 1. Runde."                                                                                                                        
[6] "So sollte das in D auch gehen!"

$...
[1] "Was mich etwas besorgt ist, dass es in Isreal (was ja sehr schnell und stark mit Einschränkungen reagiert hat) jetzt anscheinend wieder zu vermehrten Infektionen in Schulen zu kommen scheint."
[2] "Die sind jetzt wohl in einem nervigen \"auf und zu\" von Schulen."                                                                                                                              
[3] "Quelle: SZ, 5.06.20, S."                                                                                                                                                                        
[4] "16, Alexandra Föderl-Schmid, \"Schule auf, Schule zu\" Sie reagieren einerseits mit den erwähnten Schließungen und mit sog."                                                                    
[5] "Sentinels (also Stichproben in ausgewählten Einrichtungen)."                                                                                                                                    
[6] "Und mit einem Antikörper Screenings mit 70000 Personen in der 1."                                                                                                                               
[7] "Runde."                                                                                                                                                                                         
[8] "So sollte das in D auch gehen!"

> `Spacy` outperforms `tidytext` (which uses `tokenizer` package any way)

But here is an example where Deepl is better:

In [90]:
# Spacy
forum$text2[1:2]
st <- lapply(forum$text2[1:2], spacy_tokenize, what = "sentence")
st

Liebe User*innen, neben Fragen der Kinderbetreuung, Kitas oder Home-Schooling soll hier auch Platz für weitere Familienthemen wie z.B. den Umgang mit An gehörigen in Seniorenheimen sein. Hier können Sie sehen, welchen bisherigen Themen in diesem neuen Thema aufgehen Wir wünschen Ihnen spannende Diskussionen. Viele Grüße vom BRIGITTE Community-Team 
"Liebe User*innen, neben Fragen der Kinderbetreuung, Kitas oder Home-Schooling soll hier auch Platz für weitere Familienthemen wie z.B. den Umgang mit An gehörigen in Seniorenheimen sein. Hier können Sie sehen, welchen bisherigen Themen in diesem neuen Thema aufgehen Wir wünschen Ihnen spannende Diskussionen. Viele Grüße vom BRIGITTE Community-Team" 
                                                                                                                                                            Da das Thema durch den Umbau weiter nach hinten gerutscht ist, schieben wir es auf diesem Weg wieder nach vorne. Wir wünschen Ihnen spannende Diskussionen. Viele Grüße vom BRIGITTE Community-Team 
                                                                                                                                                          "Da das Thema durch den Umbau weiter nach hinten gerutscht ist, schieben wir es auf diesem Weg wieder nach vorne. Wir wünschen Ihnen spannende Diskussionen. Viele Grüße vom BRIGITTE Community-Team"

$...
$...$text1
[1] "Liebe User*innen"                                                                                                                                                        
[2] ","                                                                                                                                                                       
[3] "neben Fragen der Kinderbetreuung, Kitas oder Home-Schooling soll hier auch Platz für weitere Familienthemen wie z.B. den Umgang mit An gehörigen in Seniorenheimen sein."
[4] "Hier können Sie sehen, welchen bisherigen Themen in diesem neuen Thema aufgehen"                                                                                         
[5] "Wir wünschen Ihnen spannende Diskussionen."                                                                                                                              
[6] "Viele Grüße vom"                                                                                                                                                         
[7] "BRIGITTE Community-Team"                                                                                                                                                 


$`Da das Thema durch den Umbau weiter nach hinten gerutscht ist, schieben wir es auf diesem Weg wieder nach vorne. Wir wünschen Ihnen spannende Diskussionen. Viele Grüße vom BRIGITTE Community-Team`
$`Da das Thema durch den Umbau weiter nach hinten gerutscht ist, schieben wir es auf diesem Weg wieder nach vorne. Wir wünschen Ihnen spannende Diskussionen. Viele Grüße vom BRIGITTE Community-Team`$text1
[1] "Da das Thema durch den Umbau weiter nach hinten gerutscht ist, schieben wir es auf diesem Weg wieder nach vorne."
[2] "Wir wünschen Ihnen spannende Diskussionen."                                                                      
[3] "Viele Grüße vom"                                                                                                 
[4] "BRIGITTE Community-Team"

In [92]:
# Deepl
trans1000 <- readRDS(here("data/trans_1000.rds"))
trans1000[1:2, 2]

text
<chr>
"Dear users, in addition to questions about childcare, daycare centers or home schooling, there should also be room here for other family topics such as dealing with relatives in retirement homes. Here you can see which previous topics will be merged into this new topic We wish you exciting discussions. Best regards from the BRIGITTE Community Team"
"Since the topic has slipped further to the back due to the rebuild, we are pushing it forward again this way. We wish you exciting discussions. Best regards from the BRIGITTE Community Team"


> Decide to send paragraph to Deepl for translation! 

## API from DeepL

> Turn out it is too expensive compare to translate with app -- Sublime Text!

In [5]:
trans <- function(
    txt = "Hi", 
    key = "DeepL-Auth-Key ecb051f6-fd49-ac5a-5a96-90beb841f4d7:fx",
    url = "https://api-free.deepl.com/v2/translate", 
    in_lang = "DE", 
    out_lang = "EN"
    ) {
    out <- httr::POST(
        url = url,
        config = httr::add_headers(Authorization = key),
        body = list(text = txt, target_lang = out_lang, source_lang = in_lang)
    )
    tibble::tibble(
        code = httr::status_code(out),
        text = httr::content(out)$translations[[1]]$text
    )
}

### Translate with free API (500K)

In [23]:
# trans1000 <- map_dfr(forum$text2[1L:1000], trans)

In [24]:
forum$text2 %>%
    unname() %>%
    nchar() %>%
    tibble(n = .) %>%
    mutate(ns = cumsum(n)) %>%
    filter(ns < 499413) %>%
    nrow()
# trans1000 <- readRDS(here("data/trans1000.rds"))

[1] 1182

### Premium API

For safety, save every 1000 rows!

In [6]:
trans <- function(txt = "Hi") {
    mykey = "DeepL-Auth-Key 46cd9f89-9e38-e486-3c17-670e2333fc92"
    url = "https://api.deepl.com/v2/translate" 
    out <- httr::POST(
        url = url,
        config = httr::add_headers(Authorization = mykey),
        body = list(text = txt, target_lang = "EN", source_lang = "DE")
    )
    if (httr::status_code(out) == 200) {
        translated <- httr::content(out)$translations[[1]]$text
    } else {
        translated <- "FAIL"
    }
    tibble::tibble(code = httr::status_code(out), text = translated)
}

Named for `map` function

In [5]:
input <- forum$text2
names(input) <- forum$id

In [6]:
a <- seq(1001, length(input), 1000)
z <- c(seq(2000, length(input), 1000), length(input))
a
z

[1]  1001  2001  3001  4001  5001  6001  7001  8001  9001 10001 11001

[1]  2000  3000  4000  5000  6000  7000  8000  9000 10000 11000 11208

In [ ]:
i = 2
# tmp <- purrr::map_dfr(input[a[2]:z[2]], trans)
# head(tmp)

In [14]:
saveRDS(tmp, here("data/", "trans_1001_2000.rds"))
# saveRDS(o, here("data/", paste0(name, ".rds")))

In [ ]:
# translate and save
for (i in 1:length(a)) {
    name <- paste("trans", a[i], z[i], sep = "_")
    cat(name, '...\n')
    o <- purrr::map_dfr(input[a[i]:z[i]], trans)
    saveRDS(o, here("data/", paste0(name, ".rds")))
}

## Change to not API - cheaper

- split to txt with <1Mb file

In [13]:
nrow(forum)

[1] 11208

In [7]:
write.csv(forum$text2[1:2000], here('data', 'text2000.txt'), row.names = FALSE)

In [8]:
write.csv(forum$text2[2001:4000], here('data', 'text4000.txt'), row.names = FALSE)

In [9]:
write.csv(forum$text2[4001:6000], here('data', 'text6000.txt'), row.names = FALSE)

In [17]:
write.csv(forum$text2[6001:7500], here('data', 'text7500.txt'), row.names = FALSE)

In [19]:
write.csv(forum$text2[7501:9500], here('data', 'text9500.txt'), row.names = FALSE)

In [20]:
write.csv(forum$text2[9501:11208], here('data', 'text11208.txt'), row.names = FALSE)

These files are then drag and drop into DeepL.

## Collecting Deepl translated text

In [22]:
library(readr)
pth <- list.files(here("data"), "American", full.names = T)
pth <- pth[c(2, 3, 4, 5, 6, 1)]


In [23]:
name <- basename(pth) %>% stringr::word()
names(pth) <- name
txt <- map(pth, read_lines, skip = 1)

In [25]:
tlt <- lapply(txt, as_tibble) %>% bind_rows(.id = 'src')

In [28]:
forum <- forum %>% bind_cols(tlt)
saveRDS(forum, here('data', 'forum_tlt.rds'))